In [48]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import gridspec

## The viscous corner flow model

(Turcotte & Schubert, geodynamics, 3rd edition, ch 6.11)

In a vicous corner flow model, we consider the oceanic lithosphere as bending in a continuous manner as it is subducted at an ocean trench. 
The gravitational body force on the descending lithosphere is directed vertically downward, and pressure forces (aka, dynamic pressure) due to the induced flows in the mantle will balance the gravitational body forces.
First, the model domain is divided into the Arc corner and the Oceanic corner:

![](./figure_6_17.png)

In this analytical approach, the flow field in both the Arc corner and the Oceanic corner is solved, then the pressure forces acting on the descending litho-sphere can be calculated using these solutions.

### Define the class for solving viscous corner flow model

For the general solution, it follows from:

$$ u = -B - D arctan\left(\frac{y}{x}\right) + \left(Cx + Dy\right)\left(\frac{-x}{x^2 + y^2}\right) $$
$$ v = A + Carctan\left(\frac{y}{x}\right) + \left(Cx + Dy\right)\left(\frac{-y}{x^2 + y^2}\right) $$
$$ P = \frac{-2 \mu \left(Cx + Dy\right)}{x^2 + y^2} $$

For the arc corner:
$$ C = \frac{\theta_d sin \theta_d}{\theta_d^2 - {sin \theta_d}^2} $$
$$ D = -\frac{sin \theta_d - \theta_d cos \theta_d}{\theta_d^2 - {sin \theta_d}^2} $$
$$ A = 0 $$
$$ B = -C $$

For the oceanic corner:
$$ C = \frac{{sin \theta_d}^2 - \left(\pi - \theta_d \right)sin \theta_d}{\left(\pi - \theta_d \right)^2 - {sin \theta_d}^2}$$
$$ D = \frac{\left(\pi - \theta_d \right)\left(cos \theta_d - 1\right) + sin \theta_d - cos \theta_d sin \theta_d}{\left(\pi - \theta_d \right)^2 - {sin \theta_d}^2}$$
$$ A = -C \pi $$
$$ B = - \left(U + D \pi + C \right)


In [49]:
def GetDomain(x, y, theta_d):
    '''
    Get which domain a point belongs to
    Inputs:
        x
        y
        theta_d - dip angle
    return:
        pos - 0 (in the arc corner) or 1 (in the oceanic corner)
    '''
    theta = np.arctan2(x, y)
    pos = 0
    if theta > theta_d:
        pos = 1
    return pos


def GeneralSolution(x, y, A, B, C, D):
    '''
    Return the value of velocities from the general solution of the viscous corner flow model
    Inputs:
        x, y
        A, B, C, D - coefficents in the general solution
    Return:
        u - velocity along x
        v - velocity along y
    '''
    u = -B - D * np.arctan(y/x) + (C*x + D*y)*(-x / (x**2.0 + y**2.0))
    v = A + C * np.arctan(y/x) + (C*x + D*y)*(-y / (x**2.0 + y**2.0))
    return u, v


def PressureSolution(x, y, mu, C, D):
    '''
    Return the value of pressure from the general solution of the viscous corner flow model
    Inputs:
        x, y
        mu - viscosity
        C, D - coefficents in the general solution
    Return:
        P - pressure
    '''
    P = -2 * mu * (C * x + D * y) / (x**2.0 + y**2.0)
    return P



class VISCOUS_CORNER_FLOW():

    def __init__(self):
        '''
        Initiation
        '''
        self.theta_d = 0.0  # dip angle
        self.U = 0.0  # subducting velocity
        self.Aoc = 0.0  # coefficents for solution in the oceanic corner
        self.Boc = 0.0
        self.Coc = 0.0
        self.Doc = 0.0
        self.Aac = 0.0  # coefficents for solution in the arc corner
        self.Bac = 0.0
        self.Cac = 0.0
        self.Dac = 0.0
        self.oceanic_corner_solved = False
        self.arc_corner_solved = False

    def PrescribeDipVelocity(self, theta_d, U):
        '''
        Prescribe dip angle and velocity
        '''
        self.theta_d = theta_d
        self.U = U
    
    def PrescribeDomainViscosity(self, mu):
        '''
        Prescribe viscosity to the whole domain
        '''
        self.mu = mu

    def SolveOceanicCorner(self):
        '''
        solve coefficents in the oceanic corner
        '''
        denominator = (np.pi - theta_d)**2.0 - np.sin(theta_d)**2.0
        dividend_C = np.sin(theta_d)**2.0 - (np.pi - theta_d) * np.sin(theta_d)
        dividend_D = (np.pi - theta_d) * (np.cos(theta_d) - 1.0) + np.sin(theta_d)\
        - np.cos(theta_d) * np.sin(theta_d)
        self.Coc = dividend_C / denominator * self.U
        self.Doc = dividend_D / denominator * self.U
        self.Aoc = - self.Coc * np.pi
        self.Boc =  - (self.U + self.Doc * np.pi + self.Coc)
        self.oceanic_corner_solved = True
    
    def SolveArcCorner(self):
        '''
        solve coeffients in the arc corner
        '''
        theta_d = self.theta_d
        denominator = theta_d**2.0 - np.sin(theta_d)**2.0
        dividend_C = theta_d * np.sin(theta_d)
        dividend_D = - (np.sin(theta_d) - theta_d * np.cos(theta_d))
        self.Cac = dividend_C / denominator * self.U
        self.Dac = dividend_D / denominator * self.U
        self.Aac = 0.0 
        self.Bac = -self.Cac
        self.arc_corner_solved = True
    
    def ProblemSolved(self):
        '''
        Return whether the problem is solved
        '''
        solved = self.oceanic_corner_solved and self.arc_corner_solved\
        and self.U > 0.0 and self.mu > 0.0
        return solved
    
    def GetFlowVelocity(self, x, y):
        '''
        return flow velocity in a viscous corner flow model
        Inputs:
            x, y
            theta_d - the dip angle
        Returns:
            u - velocity along x
            v - velocity along y
            pos - 0 (in the arc corner) or 1 (in the oceanic corner)
        '''
        assert(self.ProblemSolved())  # assert solution is derived
        theta = np.arctan2(x, y)
        pos = GetDomain(x, y, self.theta_d)
        if pos == 0:
            # use the coefficients for the arc corner
            u, v = GeneralSolution(x, y, self.Aac, self.Bac, self.Cac, self.Dac)
        elif pos == 1:
            # use the coefficients for the oceanic corner
            u, v = GeneralSolution(x, y, self.Aoc, self.Boc, self.Coc, self.Doc)
        else:
            raise ValueError("A wrong value for \'pos\'")
        return u, v, pos
    
    def GetPressure(self, x, y):
        '''
        return flow velocity in a viscous corner flow model
        Inputs:
            x, y
            theta_d - the dip angle
        Returns:
            u - velocity along x
            v - velocity along y
            pos - 0 (in the arc corner) or 1 (in the oceanic corner)
        '''
        assert(self.ProblemSolved())  # assert solution is derived
        theta = np.arctan2(x, y)
        pos = GetDomain(x, y, self.theta_d)
        if pos == 0:
            P = PressureSolution(x, y, self.mu, self.Cac, self.Dac)
        elif pos == 1:
            P = PressureSolution(x, y, self.mu, self.Coc, self.Doc)
        else:
            raise ValueError("A wrong value for \'pos\'")
        return P, pos

### Generate a corner flow for a 45-degree dipping slab

In [50]:
# variable values for the model
year = 365 * 24 * 3600.0
cm_per_yr = 0.01 / year
theta_d = np.pi / 4.0
mu = 1e21
U0 =  1 * cm_per_yr
# coordinate values
x = np.linspace(-1000e3, 1000e3, 200)
y = np.linspace(0, 1000e3, 150)
xx, yy = np.meshgrid(x, y)
# initiate arrays to save values
uu = np.zeros(xx.shape)
vv = np.zeros(xx.shape)
PP = np.zeros(xx.shape)
# solve the problem
VCF = VISCOUS_CORNER_FLOW()
VCF.PrescribeDipVelocity(theta_d, U0)
VCF.PrescribeDomainViscosity(mu)
VCF.SolveArcCorner()
VCF.SolveOceanicCorner()
for i in range(xx.shape[0]):
    for j in range(xx.shape[1]):
        u, v, _ = VCF.GetFlowVelocity(xx[i, j], yy[i, j])
        P, _ = VCF.GetPressure(xx[i, j], yy[i, j])
        uu[i, j] = u
        vv[i, j] = v
        PP[i, j] = P
vv_mag = (uu**2.0 + vv**2.0) ** 0.5

### Create a function to plot

In [51]:
def PlotFieldResults(xx, yy, uu, vv, vv_mag, PP, _figsize, _title):
    '''
    Plot field vectors from the viscous corner flow model
    '''
    # Plot the solution
    fig = plt.figure(tight_layout=True, figsize=_figsize)
    gs = gridspec.GridSpec(1, 2)
    ax = fig.add_subplot(gs[0, 0])  # velocity magnitude and velocity direction
    # todo: resample the quiver, add in a stride
    h = ax.pcolormesh(xx/1e3, yy/1e3, vv_mag / cm_per_yr)
    ax.quiver(xx/1e3, yy/1e3, uu/cm_per_yr, vv/cm_per_yr, angles='xy')
    ax.invert_yaxis()
    ax.set_aspect('equal', adjustable='box')
    ax.set_xlabel('X [km]')
    ax.set_ylabel('Y [km]')
    fig.colorbar(h, ax=ax, label='Vmag (cm/yr)')
    ax = fig.add_subplot(gs[0, 1])  # pressure
    h = ax.pcolormesh(xx/1e3, yy/1e3, PP / cm_per_yr, vmin=-1e9, vmax=1e9)
    ax.invert_yaxis()
    ax.set_aspect('equal', adjustable='box')
    ax.set_xlabel('X [km]')
    ax.set_ylabel('Y [km]')
    fig.colorbar(h, ax=ax, label='Pressure (Pa)')
    fig.suptitle(_title)

# PlotFieldResults(xx, yy, uu, vv, vv_mag, PP, (15, 5), "Test Plotting Utilities")  # test